In [1]:
"""
Purpose: Creates the table that will store the soma centroid from meshes used for testing

Pseudocode: 
1) Pull down the mesh
2) Create a folder where all of the mesh pieces will be stored
3) Run the soma extraction to get the soma meshes
4) Calculate the soma centers and save the distance from the predicted soma centers
5) Things then to save off:
- index of the soma (based on tehe number of somas found for that one mesh)
- the soma center
- Distance from predicted soma center
- Mesh of the soma
6) Delete all the temporary files that were used for that certain mesh
"""

'\nPurpose: Creates the table that will store the soma centroid from meshes used for testing\n\nPseudocode: \n1) Pull down the mesh\n2) Create a folder where all of the mesh pieces will be stored\n3) Run the soma extraction to get the soma meshes\n4) Calculate the soma centers and save the distance from the predicted soma centers\n5) Things then to save off:\n- index of the soma (based on tehe number of somas found for that one mesh)\n- the soma center\n- Distance from predicted soma center\n- Mesh of the soma\n6) Delete all the temporary files that were used for that certain mesh\n'

In [2]:
# Testing 
import datajoint as dj
import numpy as np
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")
dj.config["display.limit"] = 30

import minfig
minnie = minfig.configure_minnie(return_virtual_module=True)

Connecting celiib@10.28.0.34:3306


In [3]:
import cgal_Segmentation_Module as csm
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import whole_neuron_classifier_datajoint_adapted as wcda 
import time
import trimesh
import numpy as np
import datajoint as dj
import os

In [5]:
minnie.Decimation() & (m65.AllenMultiSomas() & "status > 0").proj()

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
77489456693007645,0,0.25,218373,438232,=BLOB=
90153356676844854,0,0.25,544744,1090856,=BLOB=
90518669414951752,0,0.25,147610,296288,=BLOB=
91208269699305759,0,0.25,1550148,3100434,=BLOB=
96147481821744505,0,0.25,110154,220298,=BLOB=
107319757380005445,0,0.25,385607,774152,=BLOB=
107681564887986552,0,0.25,1455588,2922789,=BLOB=
107738877133006848,0,0.25,1361540,2746652,=BLOB=


# Helper Functions

In [7]:
def decimate_mesh_from_verts_faces(vertices,faces,segment_id,current_folder,decimation_ratio):
    #write the file to the temp folder
    input_file_base = write_Whole_Neuron_Off_file(vertices,faces,segment_id,folder=current_folder)
    output_file = input_file_base + "_decimated"
    
    
    if decimation_ratio <= 0:
        script_name = "decimation_meshlab_0_25.mls"
    else:
        script_name = f"decimation_meshlab_0_{str(decimation_ratio)[2:4]}.mls"
    meshlab_script_path_and_name = str(pathlib.Path.cwd()) + "/" + script_name


    meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                       output_path_and_filename=output_file + ".off",
                                                       meshlab_script=meshlab_script_path_and_name)
    
    #read in the output mesh and return the vertices and faces
    current_mesh = trimesh.load_mesh(output_file + '.off')
    
    #check if file exists and then delete the temporary decimated mesh filess
    if os.path.exists(input_file_base + ".off"):
        os.remove(input_file_base + ".off")
    if os.path.exists(output_file + ".off"):
        os.remove(output_file + ".off")
 
    return current_mesh.vertices,current_mesh.faces,output_file+".off"

def decimate_mesh_from_path(arg_path,decimation_ratio=0):
    #write the file to the temp folder
    input_file_base = arg_path[:-4]
    output_file = input_file_base + "_decimated"
    
    
    if decimation_ratio <= 0:
        script_name = "decimation_meshlab_0_25.mls"
    else:
        script_name = f"decimation_meshlab_0_{str(decimation_ratio)[2:4]}.mls"
    meshlab_script_path_and_name = str(pathlib.Path.cwd()) + "/" + script_name
    


    meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                       output_path_and_filename=output_file + ".off",
                                                       meshlab_script=meshlab_script_path_and_name)
    
    #read in the output mesh and return the vertices and faces
    current_mesh = trimesh.load_mesh(output_file + '.off')
    
#     #check if file exists and then delete the temporary decimated mesh filess
#     if os.path.exists(input_file_base + ".off"):
#         os.remove(input_file_base + ".off")
#     if os.path.exists(output_file + ".off"):
#         os.remove(output_file + ".off")
 
    return current_mesh,output_file+".off"

In [8]:
def decimate_mesh_from_path(input_file_path,mls_file_path):
    #write the file to the temp folder
    input_file_base = input_file_path[:-4]
    output_file = input_file_base + "_decimated"
    
    meshlab_script_path_and_name = mls_file_path
    


    meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                       output_path_and_filename=output_file + ".off",
                                                       meshlab_script=meshlab_script_path_and_name)
    
    current_mesh = trimesh.load_mesh(output_file + '.off')
    
    return current_mesh,output_file+".off"

In [9]:
import pathlib
def run_poisson_surface_reconstruction(pre_largest_mesh_path,
                                       segment_id = "None",
                                      script_name = "poisson_working_meshlab.mls"):

    """
    Will run the poisson surface reconstruction
    
    """
    # run the meshlab server script

    meshlab_script_path_and_name = str(pathlib.Path.cwd()) + "/" + script_name
    #input_path =str(pathlib.Path.cwd()) + "/" +  pre_largest_mesh_path
    input_path =  pre_largest_mesh_path

    indices = [i for i, a in enumerate(input_path) if a == "_"]
    stripped_ending = input_path[:-4]

    output_path = stripped_ending + "_mls.off"
    # print(meshlab_script_path_and_name)
    # print(input_path)
    # print(output_path)
    print("Running the mls function")
    meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_path,
                                               output_path_and_filename=output_path,
                                               segment_id=segment_id,
                                               meshlab_script=meshlab_script_path_and_name)
    return output_path

In [10]:
import os, contextlib
import pathlib
import subprocess
def meshlab_fix_manifold_path_specific_mls(input_path_and_filename,
                                           output_path_and_filename="",
                                           segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if input_path_and_filename[-4:] == ".off":
        path_and_filename = input_path_and_filename[:-4]
        input_mesh = input_path_and_filename
    else:
        raise Exception("Not passed off file")
    
    
    if output_path_and_filename == "":
        output_mesh = path_and_filename+"_mls.off"
    else:
        output_mesh = output_path_and_filename
    
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    #print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [11]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    command_to_run = 'xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command
    #command_to_run = 'meshlabserver ' + script_command
    
    print(command_to_run)
    subprocess_result = subprocess.run(command_to_run,shell=True)
    
    return subprocess_result

# Function that will extract soma

In [12]:
from pykdtree.kdtree import KDTree

import numpy as np
def validate_soma_by_sphere(current_mesh):
    ratio_val = current_mesh.bounding_sphere.volume/current_mesh.volume
    multiplier = 100
    print("Inside sphere validater: ratio_val = " + str(ratio_val))
    if np.abs(ratio_val) > multiplier or np.abs(ratio_val) < 1/multiplier:
        return False
    return True


def extract_soma_center(segment_id,current_mesh_verts,current_mesh_faces):    
    """
    Loop that will compute the soma meshes and locations from the mesh data

    """

    # ------------parameters------------------

    import time
    global_start_time = time.time()

    """
    Working for larger neuron: 


    Try 1: With 0.25 decimation
    outer_decimation_ratio= 0.25
    large_mesh_threshold = 600000
    large_mesh_threshold_inner = 40000
    soma_width_threshold = 0.32
    soma_size_threshold = 20000

    large_mesh_threshold = large_mesh_threshold*outer_decimation_ratio
    large_mesh_threshold_inner = large_mesh_threshold_inner*outer_decimation_ratio
    soma_size_threshold = soma_size_threshold*outer_decimation_ratio

    inner_decimation_ratio = 0.25
    soma_size_threshold = soma_size_threshold*inner_decimation_ratio

    Faster at: 467.977885723114 seconds


    --------------------   Try 2: Using 0.05 decimation for outer layer

    Try 1: With 0.25 decimation
    outer_decimation_ratio= 0.05
    large_mesh_threshold = 1800000
    large_mesh_threshold_inner = 40000
    soma_width_threshold = 0.32
    soma_size_threshold = 20000

    large_mesh_threshold = large_mesh_threshold*outer_decimation_ratio
    large_mesh_threshold_inner = large_mesh_threshold_inner*outer_decimation_ratio
    soma_size_threshold = soma_size_threshold*outer_decimation_ratio

    inner_decimation_ratio = 0.25
    soma_size_threshold = soma_size_threshold*inner_decimation_ratio

                    what it turned out to be ------
     large_mesh_threshold= 30000.0 
    large_mesh_threshold_inner = 2000.0 
    soma_size_threshold = 250.0\outer_decimation_ratio = 0.05\inner_decimation_ratio = 0.25


    The 0.05 decimation took almost 597 seconds

    """


    outer_decimation_ratio= 0.25
    large_mesh_threshold = 70000
    large_mesh_threshold_inner = 40000
    soma_width_threshold = 0.32
    soma_size_threshold = 20000

    large_mesh_threshold = large_mesh_threshold*outer_decimation_ratio
    large_mesh_threshold_inner = large_mesh_threshold_inner*outer_decimation_ratio
    soma_size_threshold = soma_size_threshold*outer_decimation_ratio

    inner_decimation_ratio = 0.25
    soma_size_threshold = soma_size_threshold*inner_decimation_ratio

    print(f"Current Arguments Using (adjusted for decimation):\n large_mesh_threshold= {large_mesh_threshold}"
                 f" \nlarge_mesh_threshold_inner = {large_mesh_threshold_inner} \nsoma_size_threshold = {soma_size_threshold}"
                 f"\nouter_decimation_ratio = {outer_decimation_ratio}"
                 f"\ninner_decimation_ratio = {inner_decimation_ratio}")


    # ------------------------------


    """ Create a folder and write the neuron there for all experiments to take place
    """

    import os
    folder_name = str(segment_id)
    directory = "./" + str(folder_name)
    if not os.path.exists(directory):
        os.makedirs(directory)

    full_output_mesh_name = str(pathlib.Path.cwd()) + "/" + folder_name
    mls_path = str(pathlib.Path.cwd()) + "/" + "decimation_meshlab_0_25.mls"
    
    
    base_file_path = full_output_mesh_name + "/" + folder_name + ".off"
    trimesh.Trimesh(vertices=current_mesh_verts,faces=current_mesh_faces).export(base_file_path)
    print(f"base_file_path = {base_file_path}")
    
    decimation_script_path = full_output_mesh_name


    new_mesh,decimated_output_mesh_file = decimate_mesh_from_path(base_file_path,mls_path)

    print("decimated_output_mesh_file = " + str(decimated_output_mesh_file))

    new_mesh = trimesh.load_mesh(decimated_output_mesh_file)
    
    
    
    mesh_splits = new_mesh.split(only_watertight=False)

    #len("Total mesh splits = " + str(mesh_splits))
    #get the largest mesh
    mesh_lengths = np.array([len(split.faces) for split in mesh_splits])

    # import matplotlib.pyplot as plt
    # import seaborn as sns
    # sns.set()
    # sns.distplot(mesh_lengths)

    # largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
    # largest_mesh = mesh_splits[largest_index][0]

    """ -- temporarily changing to the second largest mesh"""
    total_mesh_split_lengths = [len(k.faces) for k in mesh_splits]
    ordered_mesh_splits = mesh_splits[np.flip(np.argsort(total_mesh_split_lengths))]
    list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold]

    print(f"Total found significant pieces before Poisson = {list_of_largest_mesh}")
    
    if len(list_of_largest_mesh)<=0:
        print(f"Using smaller large_mesh_threshold because no significant pieces found with {large_mesh_threshold}")
        list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold/2]
        

    # total_soma_mesh = trimesh.Trimesh(vertices=np.array([]),
    #                                  triangles = np.array([]))

    total_soma_list = []
    total_classifier_list = []
    total_poisson_list = []

    #start iterating through where go through all pieces before the poisson reconstruction
    no_somas_found_in_big_loop = 0
    for i,largest_mesh in enumerate(list_of_largest_mesh):
        print(f"----- working on large mesh #{i}: {largest_mesh}")

        somas_found_in_big_loop = False

        stripped_ending = decimated_output_mesh_file[:-4]
        pre_largest_mesh_path = stripped_ending + "_" + str(i) + "_largest_piece.off"

        largest_mesh.export(pre_largest_mesh_path)
        print("done exporting")

        output_path = run_poisson_surface_reconstruction(pre_largest_mesh_path)

        #---------------- Will carry out the cgal segmentation -------- #
        #import the mesh
        new_mesh_inner = trimesh.load_mesh(output_path)

        mesh_splits_inner = new_mesh_inner.split(only_watertight=False)
        total_mesh_split_lengths_inner = [len(k.faces) for k in mesh_splits_inner]
        ordered_mesh_splits_inner = mesh_splits_inner[np.flip(np.argsort(total_mesh_split_lengths_inner))]
        #print(f"Mesh lengths of inner after split: {[len(k.faces) for k in ordered_mesh_splits_inner]}")

        list_of_largest_mesh_inner = [k for k in ordered_mesh_splits_inner if len(k.faces) > large_mesh_threshold_inner]
        print(f"Total found significant pieces AFTER Poisson = {list_of_largest_mesh_inner}")

        stripped_ending = output_path[:-4]
        print(f"stripped_ending 2 = {stripped_ending}")
        n_failed_inner_soma_loops = 0
        for j, largest_mesh_inner in enumerate(list_of_largest_mesh_inner):

            print(f"----- working on mesh after poisson #{j}: {largest_mesh_inner}")

            largest_mesh_path_inner = stripped_ending +"_" + str(j) + "_largest_inner.off"

            #DON'T NEED THIS WRITE NOW BECAUSE IT ALREADY OUTPUTS THE MESH
            largest_mesh_inner.export(largest_mesh_path_inner)
            print(f"done exporting {largest_mesh_path_inner}")

            largest_mesh_path_inner_decimated,decimated_output_mesh_file_inner = decimate_mesh_from_path(largest_mesh_path_inner,
                                                                       mls_path)
            largest_mesh_path_inner_decimated.export(largest_mesh_path_inner[:-4] + "_decimated.off")
            print(f"done exporting decimated mesh: {largest_mesh_path_inner}")
            # Starts the actual cgal segmentation:

            faces = np.array(largest_mesh_path_inner_decimated.faces)
            verts = np.array(largest_mesh_path_inner_decimated.vertices)
            #run the whole algorithm on the neuron to test
            segment_id_new = int(str(segment_id) + f"{i}{j}")
            verts_labels, faces_labels, soma_value,classifier = wcda.extract_branches_whole_neuron(
                                import_Off_Flag=False,
                                segment_id=segment_id_new,
                                vertices=verts,
                                 triangles=faces,
                                pymeshfix_Flag=False,
                                 import_CGAL_Flag=False,
                                 return_Only_Labels=True,
                                 clusters=3,
                                 smoothness=0.2,
                                soma_only=True,
                                return_classifier = True
                                )

            total_classifier_list.append(classifier)
            #total_poisson_list.append(largest_mesh_path_inner_decimated)

            # Save all of the portions that resemble a soma
            median_values = np.array([v["median"] for k,v in classifier.sdf_final_dict.items()])
            segmentation = np.array([k for k,v in classifier.sdf_final_dict.items()])

            #order the compartments by greatest to smallest
            sorted_medians = np.flip(np.argsort(median_values))
            print(f"segmentation[sorted_medians],median_values[sorted_medians] = {(segmentation[sorted_medians],median_values[sorted_medians])}")
            print(f"Sizes = {[classifier.sdf_final_dict[g]['n_faces'] for g in segmentation[sorted_medians]]}")

            valid_soma_segments_width = [g for g,h in zip(segmentation[sorted_medians],median_values[sorted_medians]) if ((h > soma_width_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] > soma_size_threshold))]

            print("valid_soma_segments_width")
            to_add_list = []
            if len(valid_soma_segments_width) > 0:
                print(f"      ------ Found {len(valid_soma_segments_width)} viable somas: {valid_soma_segments_width}")
                somas_found_in_big_loop = True
                #get the meshes only if signfiicant length
                labels_list = classifier.labels_list

                for v in valid_soma_segments_width:
                    submesh_face_list = np.where(classifier.labels_list == v)[0]
                    soma_mesh = largest_mesh_path_inner_decimated.submesh([submesh_face_list],append=True)
                    
                    if validate_soma_by_sphere(soma_mesh):
                        to_add_list.append(soma_mesh)
                    else:
                        print(f"--->This soma mesh was not added because it did not pass the sphere validation: {soma_mesh}")

                n_failed_inner_soma_loops = 0

            else:
                n_failed_inner_soma_loops += 1

            total_soma_list += to_add_list

            # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
            if n_failed_inner_soma_loops >= 2:
                print("breaking inner loop because 2 soma fails in a row")
                break


        # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
        if somas_found_in_big_loop == False:
            no_somas_found_in_big_loop += 1
            if no_somas_found_in_big_loop >= 2:
                print("breaking because 2 fails in a row in big loop")
                break

        else:
            no_somas_found_in_big_loop = 0



        """
        large_mesh_threshold= 150000.0 
        large_mesh_threshold_inner = 10000.0 
        soma_size_threshold = 1250.0
        decimation_ratio = 0.05
        """
    
    """ IF THERE ARE MULTIPLE SOMAS THAT ARE WITHIN A CERTAIN DISTANCE OF EACH OTHER THEN JUST COMBINE THEM INTO ONE"""
    pairings = []
    for y,soma_1 in enumerate(total_soma_list):
        for z,soma_2 in enumerate(total_soma_list):
            if y<z:
                mesh_tree = KDTree(soma_1.vertices)
                distances,closest_node = mesh_tree.query(soma_2.vertices)
                
                if np.min(distances) < 4000:
                    pairings.append([y,z])
                    
                    
    #creating the combined meshes from the list
    if len(pairings) > 0:
        """
        Pseudocode: 
        Use a network function to find components

        """
        
        
        import networkx as nx
        new_graph = nx.Graph()
        new_graph.add_edges_from(pairings)
        grouped_somas = list(nx.connected_components(new_graph))
        
        total_soma_list_revised = [] 
        somas_being_combined = []
        print(f"There were soma pariings: Connected components in = {grouped_somas} ")
        for comp in grouped_somas:
            comp = list(comp)
            somas_being_combined += list(comp)
            current_mesh = total_soma_list[comp[0]]
            for i in range(1,len(comp)):
                current_mesh += total_soma_list[comp[i]]
            
            total_soma_list_revised.append(current_mesh)
        
        #add those that weren't combined to total_soma_list_revised
        leftover_somas = [total_soma_list[k] for k in range(0,len(total_soma_list)) if k not in somas_being_combined]
        if len(leftover_somas) > 0:
            total_soma_list_revised += leftover_somas
        print(f"Final total_soma_list_revised = {total_soma_list_revised}")
        
        total_soma_list = total_soma_list_revised
            
    
    run_time = time.time() - global_start_time

    print(f"\n\n\n Total time for run = {time.time() - global_start_time}")

    #need to erase all of the temporary files ******
    #import shutil
    #shutil.rmtree(directory)
    
    """
    Need to delete all files in the temp folder *****
    """
    
    
    return total_soma_list, run_time

# Table Population

In [13]:
(minnie.Decimation() & (m65.AllenMultiSomas() & "status > 0")).proj()

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
77489456693007645,0,0.25
90153356676844854,0,0.25
90518669414951752,0,0.25
91208269699305759,0,0.25
96147481821744505,0,0.25
107319757380005445,0,0.25
107681564887986552,0,0.25
107738877133006848,0,0.25


In [29]:
decimation_version = 0
decimation_ratio = 0.25
import time


@schema
class MultiSomaCentroidValidation(dj.Computed):
    definition="""
    -> minnie.Decimation
    soma_index : tinyint unsigned #index given to this soma to account for multiple somas in one base semgnet
    ---
    centroid_x           : int unsigned                 # (EM voxels)
    centroid_y           : int unsigned                 # (EM voxels)
    centroid_z           : int unsigned                 # (EM voxels)
    distance_from_prediction : double                   # the distance of the ALLEN predicted centroid soma center from the algorithms prediction
    prediction_matching_index : int unsigned            # the soma index that was used to compute the error
    soma_vertices             : longblob                # array of vertices
    soma_faces             : longblob                   # array of faces
    multiplicity         : tinyint unsigned             # the number of somas found for this base segment
    run_time : double                   # the amount of time to run (seconds)

    """
    
    #key_source = (minnie.Decimation & (dj.U("segment_id") & (m65.AllenSegmentCentroid & "status=1").proj()) & "version=" + str(decimation_version))
    key_source = minnie.Decimation() & (m65.AllenMultiSomas() & "status > 0").proj()
    
    def make(self,key):
        #get the mesh data
        print(f"\n\n\n---- Working on {key['segment_id']} ----")
        
        new_mesh = (minnie.Decimation() & key).fetch1("mesh")
        current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
        
        segment_id = key["segment_id"]
        
        total_soma_list, run_time = extract_soma_center(segment_id,current_mesh_verts,current_mesh_faces)
        print(f"Run time was {run_time}  and the total_soma_list = {total_soma_list} and ")
        
        #check if soma list is empty and did not find soma
        if len(total_soma_list) <= 0:
            print("There were no somas found for this mesh so just writing empty data")
            insert_dict = dict(key,
                              soma_index=-1,
                              centroid_x=None,
                               centroid_y=None,
                               centroid_z=None,
                               distance_from_prediction=None,
                               prediction_matching_index = None,
                               soma_vertices=None,
                               soma_faces=None,
                               multiplicity=0,
                               run_time=run_time
                              )
            
            raise Exception("to prevent writing because none were found")
            self.insert1(insert_dict,skip_duplicates=True)
            return
        
        #if there is one or more soma found
        
        #get the soma prediction
        
        
        new_array = (m65.AllenMultiSomas.Centroids()  & key).fetch("centroid_x","centroid_y","centroid_z")
        soma_ids = (m65.AllenMultiSomas.Centroids()  & key).fetch("soma_id")
        allen_centroid_prediction = np.array(new_array).T
        allen_centroid_prediction
        #print("soma_ids = " + str(soma_ids))
        from pykdtree.kdtree import KDTree
        mesh_tree = KDTree(allen_centroid_prediction)

        
        dicts_to_insert = []
        

            
        for i,current_soma in enumerate(total_soma_list):
            print("Trying to write off file")
            current_soma.export(f"{key['segment_id']}/{key['segment_id']}_soma_{i}.off")
            auto_prediction_center = np.mean(current_soma.vertices,axis=0)
            
            distances,closest_node = mesh_tree.query(auto_prediction_center.reshape(1,3))
            error_distance = distances[0]
            prediction_matching_index = soma_ids[closest_node[0]] #closest nodes and the distances
            
            
            insert_dict = dict(key,
                              soma_index=i,
                              centroid_x=auto_prediction_center[0],
                               centroid_y=auto_prediction_center[1],
                               centroid_z=auto_prediction_center[2],
                               distance_from_prediction=error_distance,
                               prediction_matching_index = prediction_matching_index,
                               soma_vertices=current_soma.vertices,
                               soma_faces=current_soma.faces,
                               multiplicity=len(total_soma_list),
                               run_time=run_time
                              )
            
            
            
            dicts_to_insert.append(insert_dict)
            
        #raise Exception("to prevent writing")
        
        if len(total_soma_list) != len(soma_ids):
            raise Exception("to prevent writing SOMAS NOT EQUAL TO That registered in datase")
            
        self.insert(dicts_to_insert,skip_duplicates=True)

In [30]:
# new_array = (m65.AllenMultiSomas.Centroids()  & dict(segment_id=92345990027434104)).fetch("centroid_x","centroid_y","centroid_z")
# soma_ids = (m65.AllenMultiSomas.Centroids()  & dict(segment_id=92345990027434104)).fetch("soma_id")
# allen_centroid_prediction = np.array(new_array).T
# allen_centroid_prediction
# print("soma_ids = " + str(soma_ids))
# from pykdtree.kdtree import KDTree
# mesh_tree = KDTree(allen_centroid_prediction)
# distances,closest_node = mesh_tree.query([np.array([1,1,1])])
# distances[0],soma_ids[closest_node[0]] #closest nodes and the distances



In [31]:
#(SomaCentroidValidation & "segment_id=90231147459666747").delete()
(schema.jobs & "table_name='__multi_soma_centroid_validation'").delete()
#m65.SomaCentroidValidation()

In [35]:
"""
Actually will do the population
"""

#(schema.jobs & "table_name='__whole_auto_annotations_label_clusters3'")#.delete()
dj.config["enable_python_native_blobs"] = True

import time
start_time = time.time()
MultiSomaCentroidValidation.populate(reserve_jobs=True)
print(f"Total time for MultiSomaCentroidValidation populate = {time.time() - start_time}")




---- Working on 107319757380005445 ----
Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 17500.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
base_file_path = /notebooks/Platinum_Blender/Brendan_Soma/107319757380005445/107319757380005445.off
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/Brendan_Soma/107319757380005445/107319757380005445.off -o /notebooks/Platinum_Blender/Brendan_Soma/107319757380005445/107319757380005445_decimated.off -s /notebooks/Platinum_Blender/Brendan_Soma/decimation_meshlab_0_25.mls
decimated_output_mesh_file = /notebooks/Platinum_Blender/Brendan_Soma/107319757380005445/107319757380005445_decimated.off
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(92951, 3), faces.shape=(191260, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(92951, 3), faces.shape=(191260

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(39215, 3), faces.shape=(78562, 3))>, <trimesh.Trimesh(vertices.shape=(8257, 3), faces.shape=(16510, 3))>]
stripped_ending 2 = /notebooks/Platinum_Blender/Brendan_Soma/107319757380005445/107319757380005445_decimated_0_largest_piece_mls
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(39215, 3), faces.shape=(78562, 3))>
done exporting /notebooks/Platinum_Blender/Brendan_Soma/107319757380005445/107319757380005445_decimated_0_largest_piece_mls_0_largest_inner.off
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/Brendan_Soma/107319757380005445/107319757380005445_decimated_0_largest_piece_mls_0_largest_inner.off -o /notebooks/Platinum_Blender/Brendan_Soma/107319757380005445/107319757380005445_decimated_0_largest_piece_mls_0_largest_inner_decimated.off -s /notebooks/Platinum_Blender/Brendan_Soma/decimation_meshlab_0_25.mls
done exporting decimated mesh

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(117889, 3), faces.shape=(235782, 3))>, <trimesh.Trimesh(vertices.shape=(34351, 3), faces.shape=(68786, 3))>, <trimesh.Trimesh(vertices.shape=(22410, 3), faces.shape=(44820, 3))>]
stripped_ending 2 = /notebooks/Platinum_Blender/Brendan_Soma/107681564887986552/107681564887986552_decimated_0_largest_piece_mls
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(117889, 3), faces.shape=(235782, 3))>
done exporting /notebooks/Platinum_Blender/Brendan_Soma/107681564887986552/107681564887986552_decimated_0_largest_piece_mls_0_largest_inner.off
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/Brendan_Soma/107681564887986552/107681564887986552_decimated_0_largest_piece_mls_0_largest_inner.off -o /notebooks/Platinum_Blender/Brendan_Soma/107681564887986552/107681564887986552_decimated_0_largest_piece_mls_0_largest_inner_decimated.off -s /notebooks/Platinum_Ble

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(284813, 3), faces.shape=(570782, 3))>, <trimesh.Trimesh(vertices.shape=(121397, 3), faces.shape=(243354, 3))>, <trimesh.Trimesh(vertices.shape=(44644, 3), faces.shape=(89284, 3))>, <trimesh.Trimesh(vertices.shape=(44175, 3), faces.shape=(88350, 3))>, <trimesh.Trimesh(vertices.shape=(5135, 3), faces.shape=(10278, 3))>]
stripped_ending 2 = /notebooks/Platinum_Blender/Brendan_Soma/107738877133006848/107738877133006848_decimated_0_largest_piece_mls
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(284813, 3), faces.shape=(570782, 3))>
done exporting /notebooks/Platinum_Blender/Brendan_Soma/107738877133006848/107738877133006848_decimated_0_largest_piece_mls_0_largest_inner.off
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/Brendan_Soma/107738877133006848/107738877133006848_decimated_0_largest_piece_mls_0_largest_inner.off -o /notebooks/Platinum_Blen

Exception: to prevent writing SOMAS NOT EQUAL TO That registered in datase

In [ ]:
dj.config["display.limit"] = 100
m65.SomaCentroidValidation()